In [ ]:
!git clone https://github.com/jeniyat/WNUT_2020_NER

Cloning into 'WNUT_2020_NER'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (719/719), done.
remote: Compressing objects: 100% (568/568), done.
remote: Total 10058 (delta 170), reused 550 (delta 135), pack-reused 9339
Receiving objects: 100% (10058/10058), 8.92 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (5931/5931), done.


In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 4.8MB/s 
     |████████████████████████████████| 1.1MB 12.9MB/s 
     |████████████████████████████████| 890kB 29.4MB/s 
     |████████████████████████████████| 3.0MB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0f6db4f41a68a0922e064da314e7ce59af27f14e71f10873385d0333f6cb4503
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import re
import pandas as pd
def get_file_path(dirName):
    # # Get the list of all files in directory tree at given path

    # Get the list of all files in directory tree at given path
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]

    return listOfFiles
sentence = 1
def read_text_to_pd(address):
  global sentence
  # for number of sentences that is seperated with a newline 
  input_list=[]
  # schema of input list [['protocol','sentence','token','tag'],[...],...]
  protocol=0
  listOfFiles=get_file_path(address)

  for file_name in listOfFiles:
    pattern=pattern="protocol_(.*?)_conll.txt"
    # print(re.search(pattern, file_name))
    protocol = re.findall(pattern, file_name)[-1]
    # print('prot',protocol)
    protocol=int(protocol)
    
    f=open(file_name, "r")
    f1=f.readlines()
    for line in f1:
      if line=="\n":
        sentence+=1
        
        continue
      line_list=line.split("\t")
      input_list.append([protocol,sentence,line_list[0],line_list[1].replace('\n','')])
  df = pd.DataFrame(input_list, columns =['protocol', 'Sentence #','Word','Tag'])
  return df

      

tr=read_text_to_pd("/content/WNUT_2020_NER/data/train_data/Conll_Format")
dev=read_text_to_pd("/content/WNUT_2020_NER/data/dev_data/Conll_Format")
test=read_text_to_pd("/content/WNUT_2020_NER/data/test_data/Conll_Format")
l = pd.concat([tr,dev,test])

l.head()


,protocol,Sentence #,Word,Tag
0,364,1,MojoSort,B-Device
1,364,1,Human,I-Device
2,364,1,CD14,I-Device
3,364,1,Selection,I-Device
4,364,1,Kit,I-Device


In [ ]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t,p) for w,  t,p in zip(s["Word"].values.tolist(),
                                                          #  s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist(),
                                                        s["protocol"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(l)
untokenized_sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
untokenized_labels = [[s[1] for s in sentence] for sentence in getter.sentences]
print(untokenized_sentences[0])
print(untokenized_labels[0])

['MojoSort', 'Human', 'CD14', 'Selection', 'Kit', 'Protocol']
['B-Device', 'I-Device', 'I-Device', 'I-Device', 'I-Device', 'O']


In [ ]:
tag_values = list(set(l["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig,AutoTokenizer

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 200

lr = 3e-5
model_name =  "bert-base-cased"
# model_name =  "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        
        n_subwords = len(tokenized_word)
        # print(tokenized_texts)
        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(untokenized_sentences, untokenized_labels)
]
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
print(max([len(s) for s in tokenized_texts]))

250


In [ ]:
print( untokenized_labels[0])
print(untokenized_sentences[0])
ss,ll = tokenize_and_preserve_labels(untokenized_sentences[0], untokenized_labels[0])
print(ss)
print(ll)

['B-Device', 'I-Device', 'I-Device', 'I-Device', 'I-Device', 'O']
['MojoSort', 'Human', 'CD14', 'Selection', 'Kit', 'Protocol']
['Mo', '##jo', '##S', '##ort', 'Human', 'CD', '##14', 'Selection', 'Kit', 'Protocol']
['B-Device', 'B-Device', 'B-Device', 'B-Device', 'I-Device', 'I-Device', 'I-Device', 'I-Device', 'I-Device', 'O']


In [ ]:
print(len(tokenized_texts))
print(tokenized_texts[0])
print(labels[0])


14119
['Mo', '##jo', '##S', '##ort', 'Human', 'CD', '##14', 'Selection', 'Kit', 'Protocol']
['B-Device', 'B-Device', 'B-Device', 'B-Device', 'I-Device', 'I-Device', 'I-Device', 'I-Device', 'I-Device', 'O']


In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
# tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
#                                                             random_state=2018,shuffle=False, test_size=0.2)
# tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
#                                              random_state=2018,shuffle=False, test_size=0.2)

tr_inputs = torch.tensor(input_ids)
val_inputs = torch.tensor(input_ids)
tr_tags = torch.tensor(tags)
val_tags = torch.tensor(tags)
tr_masks = torch.tensor(attention_masks)
val_masks = torch.tensor(attention_masks)


# tr_inputs = torch.tensor(tr_inputs)
# val_inputs = torch.tensor(val_inputs)
# tr_tags = torch.tensor(tr_tags)
# val_tags = torch.tensor(val_tags)
# tr_masks = torch.tensor(tr_masks)
# val_masks = torch.tensor(val_masks)

In [ ]:
bs = 8
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
import transformers
from transformers import BertForTokenClassification, AdamW,AutoModelForTokenClassification

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)
model.cuda();
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=lr,
    eps=1e-8
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 5
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7423 sha256=ba98df43dd0bf3e331f846357be249d1293aa5171237f98bdbdf4f8a2abf8f17
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [ ]:
from seqeval.metrics import f1_score, accuracy_score

In [ ]:
from tqdm import tqdm, trange
import numpy as np

In [ ]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Average train loss: 0.7400765376371298
Validation loss: 0.4252614885500023
Validation Accuracy: 0.8708208962673828


Epoch:  20%|██        | 1/5 [07:37<30:30, 457.54s/it]

Validation F1-Score: 0.8395214887018166

Average train loss: 0.4476833405474428
Validation loss: 0.2973791132467369
Validation Accuracy: 0.9058909265156583


Epoch:  40%|████      | 2/5 [15:14<22:52, 457.48s/it]

...

Validation F1-Score: 0.9051285096276189

Average train loss: 0.27570217395934776
Validation loss: 0.18262850571972536
Validation Accuracy: 0.9390209408577999


Epoch:  80%|████████  | 4/5 [30:29<07:37, 457.36s/it]

Validation F1-Score: 0.9208071312693645

Average train loss: 0.22615617959530596
Validation loss: 0.16539256139579242
Validation Accuracy: 0.9438693168016026


Epoch: 100%|██████████| 5/5 [38:06<00:00, 457.29s/it]

Validation F1-Score: 0.9270536138341529



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_values, 'b-o', label="training loss")
plt.plot(validation_loss_values, 'r-o', label="validation loss")

# Label the plot.
plt.title("Learning curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
test_2020 = read_text_to_pd("/content/WNUT_2020_NER/data/test_data_2020/Conll_Format")
test_getter = SentenceGetter(test_2020)
test_sentences = [[word[0] for word in sentence] for sentence in test_getter.sentences]
test_labels = [[s[1] for s in sentence] for sentence in test_getter.sentences]
test_protocole = [[s[2] for s in sentence] for sentence in test_getter.sentences]
print(max([len(s) for s in test_sentences]))

273


In [ ]:
def predict(test_sentence):
  tokenized_sentence = tokenizer.encode(test_sentence)
  # handle bigger than 512
  init_len = len(tokenized_sentence)
  sen_init = tokenized_sentence
  if init_len > 512:
    tokenized_sentence = tokenized_sentence[:512]
  input_ids = torch.tensor([tokenized_sentence]).cuda()
  with torch.no_grad():
    output = model(input_ids)
  label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
  # join bpe split tokens
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, label_indices[0]):
      if token.startswith("##"):
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(tag_values[label_idx])
          new_tokens.append(token)
  if  init_len > 512:
    print('bigger')
    for token in tokens[512:]:
        if token.startswith("##"):
          new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
          new_labels.append('O')
          new_tokens.append(token)
  return new_labels,new_tokens

In [ ]:
def list_to_sen(list_sen):
  res = ''
  for s in list_sen:
    res += s + ' '
  return res[:-1]

In [ ]:
print(max([len(tokenizer.encode(list_to_sen(s))) for s in test_sentences]))

Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors


580


In [ ]:
print(test_sentences[0])

['Neuro-2a', 'cell', 'culture', '(', 'Neuro', '2a', ',', 'N2a', ')']


In [ ]:

for i in range(len(test_sentences)):
  # print(test_protocole[i][0])
   newlabels,newtokens = predict(list_to_sen(test_sentences[i]))
   newlabels = newlabels[1:-1]
   p = str(test_protocole[i][0])
   if len(p)<=3:
     p = '0' + p
   file_name = 'result/protocol_' + p + '_conll.txt'
  #  print(p)
   with open(file_name,mode='a') as f:
     for l,t in zip(newlabels,test_sentences[i]):
        f.write(t)
        f.write('	')
        f.write(l)
        f.write('\n')
     f.write('\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors


bigger


In [ ]:
import shutil
shutil.make_archive('result', 'zip', 'result')

'/content/result.zip'

In [ ]:
!rm -rf result
!mkdir result